# Best X finder - Test for one period

Veremos cómo funciona el algoritmo en un script distinto puesto que podría haber variables inicializadas en los test del script *two_best_X_finder.ipynb*

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap
from termcolor import colored
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import array
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [4]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 15

In [5]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
os.chdir(path)

/home/usuario/Documentos/Github/Proyectos/MLB_HN/Models/Linear_models/Free_agent/Best_X_finder


In [6]:
print("Para modificar el tamaño de todos los gráficos")
from matplotlib import rcParams
rcParams['figure.figsize'] = 15,9

Para modificar el tamaño de todos los gráficos


In [7]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Exploración de datos

Estudiaremos de manera general si hay datos faltantes, así como el tipo de datos que contiene

In [8]:
# Años de análisis
starting_year = 2012
period = 11
csv = '.csv'
# Directorios
hitters_path = 'ETL_Data/Agent/First_Year_Contract/Period_t_1/Free_Agent/Hitters/free_agents_batters_'
pitchers_path = 'ETL_Data/Agent/First_Year_Contract/Period_t_1/Free_Agent/Pitchers/free_agents_pitchers_'
# Originales:
df_pitchers = [None]*period
df_pitchers_copy = [None]*period
df_hitters = [None]*period
df_hitters_copy = [None]*period

Leamos todos las bases de datos correspondientes

In [9]:
for year in range(0,period):
    df_pitchers[year] = pd.read_csv(pitchers_path + str(starting_year + year) + csv)
    df_pitchers_copy[year] = df_pitchers[year].copy()
    
    df_hitters[year] = pd.read_csv(hitters_path + str(starting_year + year) + csv)
    df_hitters_copy[year] = df_hitters[year].copy()

Observemos el contenido de las bases de datos

In [10]:
# Pitchers
df_pitchers_copy[5].head()

,Acronimo_t,Acronimo_t_1,Altura_t,Altura_t_1,Anio_de_agente_libre_t,Anio_de_agente_libre_t_1,Anio_t,Anios_de_contrato_t,Anios_de_contrato_t_1,Antiguedad_t,...,Walks_2_t_1,Walks_t,Walks_t_1,Wins_2_t,Wins_2_t_1,Wins_t,Wins_t_1,ln_Sueldo_ajustado_t,ln_Sueldo_base_t,ln_Sueldo_regular_t
0,NYY,CHC,6.400000,6.40000,2022,2017,2017,5,1,1,...,324,20,18,16,16,4,4,16.660420,16.523561,16.660420
1,LAD,LAD,6.500000,6.50000,2022,2017,2017,5,1,0,...,121,7,11,25,9,5,3,16.243259,16.118096,16.243259
2,SF,WSH,6.100000,6.10000,2021,2017,2017,4,1,1,...,144,6,12,1,4,1,2,16.454568,15.201805,16.454568
3,LAD,LAD,6.500000,6.50000,2020,2017,2017,3,1,1,...,1089,49,33,144,144,12,12,16.354484,16.300417,16.354484
4,STL,TOR,5.718369,5.72134,2021,2017,2017,4,1,0,...,64,16,8,4,1,2,1,15.863203,15.830414,15.863203


In [11]:
list(enumerate(df_pitchers_copy[5].columns))

[(0, 'Acronimo_t'),
 (1, 'Acronimo_t_1'),
 (2, 'Altura_t'),
 (3, 'Altura_t_1'),
 (4, 'Anio_de_agente_libre_t'),
 (5, 'Anio_de_agente_libre_t_1'),
 (6, 'Anio_t'),
 (7, 'Anios_de_contrato_t'),
 (8, 'Anios_de_contrato_t_1'),
 (9, 'Antiguedad_t'),
 (10, 'Antiguedad_t_1'),
 (11, 'Bateos_en_contra_2_t'),
 (12, 'Bateos_en_contra_2_t_1'),
 (13, 'Bateos_en_contra_t'),
 (14, 'Bateos_en_contra_t_1'),
 (15, 'Bono_por_firma_t'),
 (16, 'Bono_por_firma_t_1'),
 (17, 'Cantidad_agentes_libres_t'),
 (18, 'Cantidad_agentes_libres_t_1'),
 (19, 'Cantidad_de_equipos_t'),
 (20, 'Cantidad_de_equipos_t_1'),
 (21, 'Carreras_en_contra_2_t'),
 (22, 'Carreras_en_contra_2_t_1'),
 (23, 'Carreras_en_contra_t'),
 (24, 'Carreras_en_contra_t_1'),
 (25, 'Carreras_ganadas_2_t'),
 (26, 'Carreras_ganadas_2_t_1'),
 (27, 'Carreras_ganadas_t'),
 (28, 'Carreras_ganadas_t_1'),
 (29, 'Comando_2_t'),
 (30, 'Comando_2_t_1'),
 (31, 'Comando_t'),
 (32, 'Comando_t_1'),
 (33, 'Control_2_t'),
 (34, 'Control_2_t_1'),
 (35, 'Control_t'),
 

In [12]:
# Hitters
df_hitters_copy[5].head()

,Acronimo_t,Acronimo_t_1,Altura_t,Altura_t_1,Anio_de_agente_libre_t,Anio_de_agente_libre_t_1,Anio_t,Anios_de_contrato_t,Anios_de_contrato_t_1,Antiguedad_t,...,Victorias_t_1,WAR_2_t,WAR_2_t_1,WAR_t,WAR_t_1,WS_ganadas_t,WS_ganadas_t_1,ln_Sueldo_ajustado_t,ln_Sueldo_base_t,ln_Sueldo_regular_t
0,NYM,NYM,5.718369,5.72134,2021,2017,2017,4,3,0,...,87,3.841600,8.7025,1.960000,2.95,2,2,16.929026,16.929026,16.929026
1,STL,CHC,6.500000,6.50000,2022,2017,2017,5,1,1,...,102,3.097600,15.9201,1.760000,3.99,11,3,16.618871,16.489659,16.618871
2,COL,TEX,6.300000,6.30000,2023,2017,2017,5,1,0,...,95,3.009561,4.8841,1.734809,2.21,0,0,15.894952,15.894952,15.894952
3,HOU,LAD,6.200000,6.20000,2021,2017,2017,4,1,1,...,91,20.250000,0.0016,4.500000,0.04,1,6,16.380460,16.380460,16.380460
4,LAD,LAD,6.500000,6.50000,2020,2017,2017,3,1,1,...,91,4.884100,1.9321,2.210000,1.39,6,6,16.354484,16.300417,16.354484


Veamos las columnas con sus índices

In [13]:
list(enumerate(df_hitters_copy[5].columns))

[(0, 'Acronimo_t'),
 (1, 'Acronimo_t_1'),
 (2, 'Altura_t'),
 (3, 'Altura_t_1'),
 (4, 'Anio_de_agente_libre_t'),
 (5, 'Anio_de_agente_libre_t_1'),
 (6, 'Anio_t'),
 (7, 'Anios_de_contrato_t'),
 (8, 'Anios_de_contrato_t_1'),
 (9, 'Antiguedad_t'),
 (10, 'Antiguedad_t_1'),
 (11, 'At_bats_2_t'),
 (12, 'At_bats_2_t_1'),
 (13, 'At_bats_t'),
 (14, 'At_bats_t_1'),
 (15, 'Bateos_2_t'),
 (16, 'Bateos_2_t_1'),
 (17, 'Bateos_promedio_2_t'),
 (18, 'Bateos_promedio_2_t_1'),
 (19, 'Bateos_promedio_t'),
 (20, 'Bateos_promedio_t_1'),
 (21, 'Bateos_t'),
 (22, 'Bateos_t_1'),
 (23, 'Bono_por_firma_t'),
 (24, 'Bono_por_firma_t_1'),
 (25, 'Cantidad_agentes_libres_t'),
 (26, 'Cantidad_agentes_libres_t_1'),
 (27, 'Cantidad_de_equipos_t'),
 (28, 'Cantidad_de_equipos_t_1'),
 (29, 'Dobles_2_t'),
 (30, 'Dobles_2_t_1'),
 (31, 'Dobles_t'),
 (32, 'Dobles_t_1'),
 (33, 'Edad_al_firmar_t'),
 (34, 'Edad_al_firmar_t_1'),
 (35, 'Edad_t'),
 (36, 'Equipo_anterior'),
 (37, 'Equipo_t'),
 (38, 'Equipo_t_1'),
 (39, 'Estado_t'),
 

Veamos si hay bases de datos con variables que distintas entre sí

## Test

In [14]:
def etl_regression(X_raw, Y_raw, df_copy):
    # Creación de la base de datos a transformar las variables de la regresión
    jugador = 'Jugador'
    YX_raw = [None]*period
    
    for year in range(0, period):
        YX_raw[year] = df_copy[year][[jugador, Y_raw, X_raw]]

    # Ajuste de inflación en los salarios    
    # Ajuste de inflación en los salarios    
    cpi = {2012:1.31, 2013:1.29,
           2014:1.27, 2015:1.27,
           2016:1.26, 2017:1.23,
           2018:1.20, 2019:1.18,
           2020:1.15, 2021:1.14,
           2022:1}
    
    for year in range(0, period):
        YX_raw[year][[Y_raw]] = YX_raw[year][[Y_raw]]*cpi[starting_year + year]
        YX_raw[year].rename(columns = {Y_raw: Y_raw + '_' + str(2012 + year)}, inplace = True)
        YX_raw[year].rename(columns = {X_raw: X_raw + '_' + str(2012 + year)}, inplace = True)

    # Filtrado de base de datos
    YX_panel_raw = pd.merge(YX_raw[0], YX_raw[1], how = 'outer', on = 'Jugador')

    for year in range(2,period):
        YX_panel_raw = pd.merge(YX_panel_raw, YX_raw[year], how = 'outer', on = 'Jugador')

    YX_panel_raw.drop_duplicates(subset = 'Jugador', inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw.sort_index(axis = 1, inplace = True)

    # Recolocación al principio de Jugador
    player_column = YX_panel_raw.pop('Jugador')
    YX_panel_raw.insert(0, 'Jugador', player_column)

    # Creación de base de datos auxiliares  para remover los NaN
    split_column = 1 + period
    max_nan = 2

    if Y_raw > X_raw:
        X_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]

    nulls_quantity = period - max_nan
    X_drop_etl = X_drop_etl.iloc[X_drop_etl[(X_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    X_index_drop = X_drop_etl.index

    if Y_raw > X_raw:
        Y_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    Y_drop_etl = Y_drop_etl.iloc[Y_drop_etl[(Y_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    Y_index_drop = Y_drop_etl.index

    # índice de los jugadores que no tienen más de dos observaciones en el periodo de analisis
    index_drop = list(set(X_index_drop).union(set(Y_index_drop)))

    YX_panel_raw.drop(index = index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)

    # Borremos provicionalmente a los jugadores cuyo estadística máxima es igual a 0
    if Y_raw > X_raw:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, 1:split_column].max(axis = 1)
    else:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]].max(axis = 1)
    YX_max_0 = YX_panel_raw[YX_panel_raw[X_raw + '_max'] == 0]
    YX_max_0_index_drop = YX_max_0.index
    YX_panel_raw.drop(columns = X_raw + '_max', inplace = True)
    YX_panel_raw.drop(index = YX_max_0_index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)

    # Variable X
    if Y_raw > X_raw:
        X_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    max_element = X_etl.max(axis = 1)
    min_element = X_etl.min(axis = 1)
    max_min = (max_element + min_element)/2
    dummy_names = []
    X_names = []
    for year in range(0,period):
        dummy_names.append("I_" + str(2012 + year))
        X_names.append("X_" + str(2012 + year))
    for year in range(0,period):
        X_conditions = [
        (X_etl.iloc[:,year] == max_element) & ~(X_etl.iloc[:,year].isnull()),
        (X_etl.iloc[:,year] != max_element) & ~(X_etl.iloc[:,year].isnull())
        ]
        X_conditions_values = [0,1]
        X_etl[dummy_names[year]] = np.select(X_conditions, X_conditions_values, default = np.nan)
    X_aux = X_etl.iloc[:,:period]
    Dummy = X_etl.iloc[:,period:]
    X_max = X_aux.max(axis = 1)
    X_aux[['Max']] = np.sqrt(X_max)
    for year in range(0,period):    
        X_etl['X_' + str(2012 + year)] = \
        (X_aux[X_raw + '_' + str(2012 + year)]*
        ((-1)**Dummy['I_' + str(2012 + year)]))
    X_etl = X_etl.div(X_aux['Max'].values, axis = 0)
    X_etl = X_etl.iloc[:,2*period:]
    for i in range(0,X_etl.shape[0]):
        row = X_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            X_etl.iloc[i,j] = sustitute[j]

    X_etl.columns = ['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 
                     'X_7', 'X_8', 'X_9', 'X_10', 'X_11']
    X_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    X_player_column = X_etl.pop('Jugador')
    X_etl.insert(0, 'Jugador', X_player_column)

    # Variable Y
    if Y_raw > X_raw:
        Y_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_etl = YX_panel_raw.iloc[:, 1:split_column]
    for i in range(0,Y_etl.shape[0]):
        row = Y_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl.iloc[i,j] = sustitute[j]

    Y_etl.columns = ['w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 
                     'w_7', 'w_8', 'w_9', 'w_10', 'w_11']
    Y_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    Y_player_column = Y_etl.pop('Jugador')
    Y_etl.insert(0, 'Jugador', Y_player_column)
    Y_etl_copy = Y_etl.copy()
    y_etl_names = Y_etl_copy['Jugador']
    for i in range(0,len(y_etl_names)):
        row = Y_etl_copy.iloc[i]
        row_length = row.size
        k = 0

        sustitute = []

        for j in range(0, row_length):
            if pd.isna(row[j]) != True:
                k = k + 1

        sustitute.append(y_etl_names[i])
        for j in range(1,k - 1):
            sustitute.append(row[j + 1]**0.5 - row[j]**0.5)

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl_copy.iloc[i,j] = sustitute[j]

    Y_etl_copy.columns = ['Jugador', 'Y_1', 'Y_2', 'Y_3', 'Y_4', 'Y_5', 'Y_6', 
                          'Y_7', 'Y_8', 'Y_9', 'Y_10', 'Y_11']
    Y_etl = Y_etl_copy

    # Base de datos
    regression_list = [None]*(period-1)
    for i in range(0,len(regression_list)):
        merge = pd.merge(Y_etl.iloc[:,[0,i+1]], X_etl.iloc[:,[0,i+1]])
        regression_list[i] = merge
        regression_list[i].columns = ['Jugador', 'Y', 'X']
    df_article_regression = regression_list[0]
    for i in range(1,len(regression_list)):
        df_article_regression = pd.concat([df_article_regression, regression_list[i]])
    df_article_regression = df_article_regression.sort_values(by = 'Jugador', ascending = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    df_article_regression.dropna(inplace = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    
    # Construimos la regresion
    Y = df_article_regression['Y'].tolist()
    X = df_article_regression['X'].tolist()
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    
    # Regresamos como argumento al modelo
    return [df_article_regression, model]

In [15]:
X_raw_pitcher_t = 'Inning_pitched_t_1'
Y_raw_t = 'Sueldo_ajustado_t'
df_copy_t = df_pitchers_copy

In [16]:
df_regression_t, model_t = etl_regression(X_raw_pitcher_t, Y_raw_t, df_copy_t)

Veamos la base de datos ocupada para la regresión y después los datos sobre la regresión

In [17]:
df_regression_t

,Jugador,Y,X
0,A.J. Burnett,-494.306419,-13.065653
1,Adam Ottavino,-1022.766170,8.814760
2,Adam Wainwright,1174.017349,13.103435
3,Alex Cobb,0.000000,13.390295
4,Alex Colome,-362.621604,-2.765975
...,...,...,...
168,Yusmeiro Petit,-200.065952,9.555103
169,Zach Duke,1351.616723,-4.085313
170,Zach Duke,711.764513,-1.788140
171,Zack Greinke,-2939.675993,14.923136


In [18]:
model_t.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     15.50
Date:                Thu, 12 Jan 2023   Prob (F-statistic):           0.000120
Time:                        15:04:22   Log-Likelihood:                -1480.5
No. Observations:                 173   AIC:                             2965.
Df Residuals:                     171   BIC:                             2971.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         57.7843     96.518      0.599      0.550    -132.736     248.305
x1           -44.4994     11.302     -3.937      0.000     -66.809     -22.190
==============================================================================
Omnibus:                        3.746   Durbin-Watson:                   2.192
Prob(Omnibus):                  0.154   Jarque-Bera (JB):                3.734
Skew:                          -0.356   Prob(JB):                        0.155
Kurtosis:                       2.893   Cond. No.                         8.55
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Vemos que el algoritmo funciona adecuadamente

## Pitchers

Hagamos una prueba con una serie de estadísticas deportivas para cada grupo

In [19]:
pitcher_statistics_t = ['Bateos_en_contra_t_1', 'Carreras_en_contra_t_1', 'Carreras_ganadas_t_1', 'Comando_t_1', 'Control_t_1',
                        'Dominio_t_1', 'ERA_t_1', 'Inning_pitched_t_1', 'Juegos_t_1', 'Juegos_iniciados_t_1', 
                        'Losses_t_1', 'Saves_t_1', 'TVS_t_1', 'WAR_t_1', 'WHIP_t_1', 'Walks_t_1', 'Wins_t_1', 
                        'Bateos_en_contra_2_t_1', 'Carreras_en_contra_2_t_1', 'Carreras_ganadas_2_t_1', 'Comando_2_t_1', 'Control_2_t_1',
                        'Dominio_2_t_1', 'ERA_2_t_1', 'Inning_pitched_2_t_1', #'Juegos_iniciados_2_t_1', 
                        'Losses_2_t_1', 'Saves_2_t_1', 'TVS_2_t_1', 'WAR_2_t_1', 'WHIP_2_t_1', 'Walks_2_t_1', 'Wins_2_t_1']
Y_pitcher_raw_t = 'Sueldo_ajustado_t'
pitcher_Statistics_len = len(pitcher_statistics_t)
pitcher_model_t = [None]*pitcher_Statistics_len
df_pitcher_model_t = [None]*pitcher_Statistics_len

In [20]:
for statistic in range(0,pitcher_Statistics_len):
    df_pitcher_model_t[statistic], pitcher_model_t[statistic] = etl_regression(pitcher_statistics_t[statistic],
                                                                               Y_pitcher_raw_t,
                                                                               df_pitchers_copy)
    
    print(colored(pitcher_statistics_t[statistic], "cyan"))
    print(pitcher_model_t[statistic].summary())
    print('\n')

Bateos_en_contra_t_1
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     4.924
Date:                Thu, 12 Jan 2023   Prob (F-statistic):             0.0278
Time:                        15:04:24   Log-Likelihood:                -1485.6
No. Observations:                 173   AIC:                             2975.
Df Residuals:                     171   BIC:                             2982.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         33.8042     99.24

## Hitters

Hagamos lo mismo para los *hitters*.

In [23]:
hitter_statistics_t = ['At_bats_t_1', 'Bateos_t_1', 'Home_runs_t_1', 'Juegos_t_1',
                       'Porcentaje_On_base_plus_slugging_t_1', 'Bateos_promedio_t_1', 'Porcentaje_juegos_t_1',
                       'Runs_batted_in_t_1', 'TVS_t_1', 'WAR_t_1', 'At_bats_2_t_1', 'Bateos_2_t_1',
                       'Home_runs_2_t_1', 'Juegos_2_t_1', 'Porcentaje_On_base_plus_slugging_2_t_1',
                       'Bateos_promedio_2_t_1', 
                       'Runs_batted_in_2_t_1', 'TVS_2_t_1', 'WAR_2_t_1']
Y_hitter_raw_t = 'Sueldo_ajustado_t'
hitter_Statistics_len = len(hitter_statistics_t)
hitter_model_t = [None]*hitter_Statistics_len
df_hitter_model_t = [None]*hitter_Statistics_len

In [24]:
for statistic in range(0,hitter_Statistics_len):
    df_hitter_model_t[statistic], hitter_model_t[statistic] = etl_regression(hitter_statistics_t[statistic],
                                                                             Y_hitter_raw_t,
                                                                             df_hitters_copy)
    
    print(colored(hitter_statistics_t[statistic], "cyan"))
    print(hitter_model_t[statistic].summary())
    print('\n')

At_bats_t_1
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                   0.02420
Date:                Thu, 12 Jan 2023   Prob (F-statistic):              0.877
Time:                        15:05:27   Log-Likelihood:                -1319.1
No. Observations:                 153   AIC:                             2642.
Df Residuals:                     151   BIC:                             2648.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -204.2906    110.337     -1.

KeyError: "['Juegos_2_t_1'] not in index"